In [14]:
#데이터 준비
import yfinance 
import numpy as np


In [15]:
df = yfinance.download('AAPL','2000-1-1','2020-1-1')
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,0.936384,1.004464,0.907924,0.999442,0.847207,535796800
2000-01-04,0.966518,0.987723,0.903460,0.915179,0.775779,512377600
2000-01-05,0.926339,0.987165,0.919643,0.928571,0.787131,778321600
2000-01-06,0.947545,0.955357,0.848214,0.848214,0.719014,767972800
2000-01-07,0.861607,0.901786,0.852679,0.888393,0.753073,460734400
...,...,...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497,69.236229,48478800
2019-12-26,71.205002,72.495003,71.175003,72.477501,70.609894,93121200
2019-12-27,72.779999,73.492500,72.029999,72.449997,70.583107,146266000


In [6]:
#df = df.drop(['Volume'],1).drop(['Adj Close'],1)

TypeError: DataFrame.drop() takes from 1 to 2 positional arguments but 3 were given

In [16]:
df = yfinance.download('AAPL','2000-1-1','2020-1-1').drop(['Volume','Adj Close'], axis=1)
df
# 여기서 volume and Adj 삭제 

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2000-01-03,0.936384,1.004464,0.907924,0.999442
2000-01-04,0.966518,0.987723,0.903460,0.915179
2000-01-05,0.926339,0.987165,0.919643,0.928571
2000-01-06,0.947545,0.955357,0.848214,0.848214
2000-01-07,0.861607,0.901786,0.852679,0.888393
...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497
2019-12-26,71.205002,72.495003,71.175003,72.477501
2019-12-27,72.779999,73.492500,72.029999,72.449997


In [17]:
#함수 적용 
#데이터 프레임 내에서 데이터 세트를 정규화하기 위한 포괄적인 함수임. 
#모든 열의 이름을 숫자로 바꾸는 일반적인 방법에 유의할 것. 
#데이터 프레임의 크기에 관계없이 적용될 수 있음.add()

def normalize_data(dataset): 
    cols = dataset.columns.tolist()
    col_name = [0]*len(cols)
    for i in range(len(cols)):
        col_name[i] = i
    dataset.columns = col_name
    dtypes = dataset.dtypes.tolist()
    minmax = list()
    
    for column in dataset: 
        dataset = dataset.astype({column: 'float32'})
    for i in range(len(cols)):
        col_values = dataset[col_name[i]]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min,value_max])
    for column in dataset:
        values = dataset[column].values
        for i in range(len(values)):
            values[i] = (values[i] - minmax[column][0])/ (minmax[column][1]-minmax[column][0])
            dataset[column][0] = values
            dataset[column] = values
            return dataset, minmax 
        
        dataset,minmax = normalize_data(df)
        print(df.values)
        values= dataset.values

In [19]:
# 시퀀스를 데이터 세트로 변환
#X값은 종가를 제외한 다른 모든 열이며 이는 y값이 된다. 
# 이 부분이 컴퓨터가 종가를 예측하기 위해 주식 시세를 사용할 것임을 의미함.

#시퀀스를 데이터 세트로 변환하는 함수 
def split_sequences(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)): 
        end_ix = i + n_steps
        if end_ix> len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

#데이터를 설정하는 함수 
def data_setup(n_steps, n_seq,sequence):
    X,y = split_sequences(sequence, n_steps)
    n_features = X.shape[2]
    X= X.reshape((len(X),n_steps, n_features))
    new_y= []
    for term in y:
        new_term = term[-1]
        new_y.append(new_term)
    return X, np.array(new_y), n_features

#주어진 값 
values =np.random.rand(100,5)    #임의의 시퀀스 생성 

#데이터 설정 
n_steps = 10
n_seq = 10000
rel_test_len = 0.1
X,y,n_features = data_setup(n_steps,n_seq,values)

#데이터 분할 
X= X[:-1]
y=y[1:]
X_test,y_test =X[:int(len(X)*rel_test_len)],y[:int(len(X)*rel_test_len)]
X_train,y_train = X[int(len(X)*rel_test_len):],y[int(len(X)*rel_test_len):]


#결과 확인
print(X.shape)


#X.shape에서 나옴 
#첫 번째 차원은 데이터 포인트의 수입니다. 즉, 데이터 세트에는 총 89개의 데이터 포인트가 있습니다.
#두 번째 차원은 시퀀스의 길이 또는 타임 스텝의 수를 나타냅니다. 여기서는 각 데이터 포인트가 10개의 타임 스텝을 가지고 있음을 의미합니다.
#세 번째 차원은 각 타임 스텝에서의 특성의 수입니다. 데이터 세트에는 각 타임 스텝에서 5개의 특성이 있습니다.
#따라서 (89, 10, 5) 형태의 X는 총 89개의 데이터 포인트가 있고, 각각의 데이터 포인트는 10개의 타임 스텝으로 구성되며, 각 타임 스텝은 5개의 특성을 가지고 있음을 나타냅니다.

#이 코드는 주식 시세를 예측하기 위해 데이터를 준비하고 있습니다. 
# 따라서 각 데이터 포인트는 일련의 주식 가격 정보를 나타내며, 각 타임 스텝에서 5개의 특성(예: 시가, 고가, 저가, 거래량 등)이 포함됩니다.






(89, 10, 5)


In [ ]:
#LSTM 

In [26]:
! uninstall keras

In [24]:
!pip install tensorflow

     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/65.5 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.5 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.5 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.5 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.5 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.5 kB ? eta -:--:--
     ----------------------------------- -- 61.4/65.5 kB 204.8 kB/s eta 0:00:01
     -------------------------------------- 65.5/65.5 kB 207.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---- ----------------------------------- 0.2/1.5 MB 3.1 MB/s eta 0:00:01
     ------------ --------------------------- 0.5/1.5 MB 5.0 MB/s eta 0:00:01
     -------------------